<a href="https://colab.research.google.com/github/ShilpaSangappa/END-NLP-Phase1/blob/main/session_4_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

from torchtext import data
from torchtext import datasets

In [3]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy', include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)

In [4]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:06<00:00, 12.3MB/s]


In [5]:
for i in range(len(train_data)):
  train_data.examples[i].text.reverse()

In [26]:
for i in range(len(test_data)):
  test_data.examples[i].text.reverse()

In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 399207/400000 [00:23<00:00, 17056.06it/s]

In [27]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, sort_within_batch = True, device = device)


In [9]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout, pad_idx):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
    self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional=bidirectional, dropout=dropout)
    self.fc = nn.Linear(hidden_dim*2, output_dim)
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, text, text_lengths):
    embedded = self.dropout(self.embedding(text))
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
    packed_output, (hidden, cell) = self.rnn(packed_embedded)
    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

    hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))

    return self.fc(hidden)
  


In [10]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = False
DROPOUT = 0.2

PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [11]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')  

The model has 3,919,977 trainable parameters


In [12]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [13]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])

In [14]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3970,  0.4024,  1.0612,  ..., -0.0136, -0.3363,  0.6442],
        [-0.5197,  1.0395,  0.2092,  ..., -0.8857, -0.2294,  0.1244],
        [ 0.0057, -0.0707, -0.0804,  ..., -0.3292, -0.0130,  0.0716]])


In [15]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [16]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [17]:
def binary_accuracy(preds, y):
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)
  return acc

In [18]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0

  model.train()

  for batch in iterator:
    optimizer.zero_grad()
    text, text_lengths = batch.text
    predictions = model(text, text_lengths.cpu()).squeeze(1)
    loss = criterion(predictions, batch.label)
    acc = binary_accuracy(predictions, batch.label)
    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()
  
  return epoch_loss / len(iterator), epoch_acc/len(iterator)

In [19]:
def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0

  model.eval()

  with torch.no_grad():
    for batch in iterator:
      text, text_lengths = batch.text
      predictions = model(text, text_lengths.cpu()).squeeze(1)
      loss = criterion(predictions, batch.label)
      acc = binary_accuracy(predictions, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()
  return epoch_loss / len(iterator), epoch_acc/len(iterator)

In [20]:
import time

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [21]:
N_EPOCHS = 35

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
  start_time = time.time()

  train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'tut2-model-epoch25.pt')
  
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 18s
	Train Loss: 0.674 | Train Acc: 57.55%
	 Val. Loss: 0.704 |  Val. Acc: 49.97%
Epoch: 02 | Epoch Time: 1m 18s
	Train Loss: 0.646 | Train Acc: 62.30%
	 Val. Loss: 0.684 |  Val. Acc: 56.83%
Epoch: 03 | Epoch Time: 1m 18s
	Train Loss: 0.515 | Train Acc: 75.50%
	 Val. Loss: 0.490 |  Val. Acc: 77.53%
Epoch: 04 | Epoch Time: 1m 17s
	Train Loss: 0.327 | Train Acc: 86.72%
	 Val. Loss: 0.335 |  Val. Acc: 86.32%
Epoch: 05 | Epoch Time: 1m 18s
	Train Loss: 0.261 | Train Acc: 89.93%
	 Val. Loss: 0.322 |  Val. Acc: 87.19%
Epoch: 06 | Epoch Time: 1m 17s
	Train Loss: 0.219 | Train Acc: 91.86%
	 Val. Loss: 0.336 |  Val. Acc: 87.67%
Epoch: 07 | Epoch Time: 1m 17s
	Train Loss: 0.182 | Train Acc: 93.51%
	 Val. Loss: 0.333 |  Val. Acc: 87.80%
Epoch: 08 | Epoch Time: 1m 17s
	Train Loss: 0.138 | Train Acc: 95.38%
	 Val. Loss: 0.344 |  Val. Acc: 87.60%
Epoch: 09 | Epoch Time: 1m 17s
	Train Loss: 0.109 | Train Acc: 96.42%
	 Val. Loss: 0.354 |  Val. Acc: 86.60%
Epoch: 10 | Epoch T

In [22]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss : {test_loss:.3f} | Test Acc: {test_acc*100:.3f}%')

Test Loss : 0.745 | Test Acc: 84.675%


In [25]:
model.load_state_dict(torch.load('tut2-model-epoch25.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss : {test_loss:.3f} | Test Acc: {test_acc*100:.3f}%')

Test Loss : 0.345 | Test Acc: 85.810%


In [28]:
#After reversing text in 'test_data' as well
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss : {test_loss:.3f} | Test Acc: {test_acc*100:.3f}%')

Test Loss : 0.331 | Test Acc: 86.537%
